In [229]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import MolStandardize

from IPython.display import display, Markdown, HTML


In [230]:
# Проверяет, можно ли построить молекулу из SMILES
def is_valid_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False
    
# Приведение молекулы в в каноническую форму
def canonicalize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol, canonical=True)

# Проверяет, содержит ли молекула несколько фрагментов (соли, смеси и т.п.)    
def has_multiple_fragments(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return False
        # считаем количество независимых фрагментов
        frags = Chem.GetMolFrags(mol)
        return len(frags) > 1
    except:
        return False     






In [231]:
#Канцерогенность — это способность веществ или факторов вызывать или повышать вероятность развития рака
df = pd.read_csv('../datasets/toxric_30_datasets/Carcinogenicity_Carcinogenicity.csv')
display(Markdown("### Канцерогенность:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value')


### Канцерогенность:

Размерность: (1021, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021 entries, 0 to 1020
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              1021 non-null   object 
 1   Name              541 non-null    object 
 2   IUPAC Name        1002 non-null   object 
 3   PubChem CID       1002 non-null   float64
 4   Canonical SMILES  1009 non-null   object 
 5   InChIKey          1001 non-null   object 
 6   Toxicity Value    1021 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 56.0+ KB
None


#### Валидность молекул:

Корректных молекул: 1009
Некорректных молекул: 12
Доля невалидных: 1.18%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
31,TOX-530,NaN,NaN,NaN,NaN,NaN,0,False
109,TOX-1849,NaN,NaN,NaN,NaN,NaN,0,False
135,TOX-2231,NaN,NaN,NaN,NaN,NaN,0,False
173,TOX-2941,NaN,NaN,NaN,NaN,NaN,0,False
184,TOX-3152,NaN,NaN,NaN,NaN,NaN,0,False
217,TOX-4020,NaN,NaN,NaN,NaN,NaN,0,False
220,TOX-4048,NaN,NaN,NaN,NaN,NaN,0,False
242,TOX-4318,NaN,NaN,NaN,NaN,NaN,0,False
368,TOX-5772,NaN,NaN,NaN,NaN,NaN,0,False
472,TOX-9178,NaN,NaN,NaN,NaN,NaN,0,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts


#### Пропуски:

TAID                  0
Name                480
IUPAC Name           19
PubChem CID          19
Canonical SMILES     12
InChIKey             20
Toxicity Value        0
is_valid              0
has_salts             0
dtype: int64
TAID                 0.000000
Name                47.012733
IUPAC Name           1.860921
PubChem CID          1.860921
Canonical SMILES     1.175318
InChIKey             1.958864
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 49.66%


In [232]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Carcinogenicity_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Carcinogenicity_Carcinogenicity.csv', index=False)


In [233]:
# "Mutagenicity Ames" относится к тесту Эймса, который является генетическим тестом для оценки мутагенного потенциала химических веществ. 
df = pd.read_csv('../datasets/toxric_30_datasets/Mutagenicity_Ames Mutagenicity.csv')
display(Markdown("### Мутагенность:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value')

### Мутагенность:

Размерность: (7485, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7485 entries, 0 to 7484
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              7485 non-null   object 
 1   Name              1975 non-null   object 
 2   IUPAC Name        6870 non-null   object 
 3   PubChem CID       6889 non-null   float64
 4   Canonical SMILES  7418 non-null   object 
 5   InChIKey          6878 non-null   object 
 6   Toxicity Value    7485 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 409.5+ KB
None


[17:42:35] Explicit valence for atom # 3 N, 5, is greater than permitted
[17:42:35] Explicit valence for atom # 1 N, 5, is greater than permitted


#### Валидность молекул:

Корректных молекул: 7416
Некорректных молекул: 69
Доля невалидных: 0.92%


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts


#### Пропуски:

TAID                   0
Name                5510
IUPAC Name           615
PubChem CID          596
Canonical SMILES      67
InChIKey             607
Toxicity Value         0
is_valid               0
has_salts              0
dtype: int64
TAID                 0.000000
Name                73.613894
IUPAC Name           8.216433
PubChem CID          7.962592
Canonical SMILES     0.895124
InChIKey             8.109552
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 56.06%


In [234]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
df["Canonical SMILES"] = df["Canonical SMILES"].apply(canonicalize_smiles)

# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Mutagenicity_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Mutagenicity_Ames Mutagenicity.csv', index=False)


[17:42:36] Explicit valence for atom # 3 N, 5, is greater than permitted
[17:42:36] Explicit valence for atom # 1 N, 5, is greater than permitted


 Clinical toxicity (клиническая токсичность) относится к вредным, токсическим эффектам веществ, наблюдаемым непосредственно у человека

In [235]:

df = pd.read_csv('../datasets/toxric_30_datasets/Clinical Toxicity_Clinical toxicity.csv')
display(Markdown("### Клиническая токсичность:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value')

### Клиническая токсичность:

Размерность: (1430, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1430 entries, 0 to 1429
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              1430 non-null   object 
 1   Name              749 non-null    object 
 2   IUPAC Name        1241 non-null   object 
 3   PubChem CID       1243 non-null   float64
 4   Canonical SMILES  1415 non-null   object 
 5   InChIKey          1243 non-null   object 
 6   Toxicity Value    1430 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 78.3+ KB
None


#### Валидность молекул:

Корректных молекул: 1415
Некорректных молекул: 15
Доля невалидных: 1.05%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
25,TOX-27,NaN,NaN,NaN,NaN,NaN,0,False
115,TOX-121,NaN,NaN,NaN,NaN,NaN,1,False
519,TOX-530,NaN,NaN,NaN,NaN,NaN,0,False
884,TOX-901,NaN,NaN,NaN,NaN,NaN,0,False
907,TOX-924,NaN,NaN,NaN,NaN,NaN,0,False
909,TOX-926,NaN,NaN,NaN,NaN,NaN,0,False
910,TOX-927,NaN,NaN,NaN,NaN,NaN,0,False
911,TOX-928,NaN,NaN,NaN,NaN,NaN,0,False
912,TOX-929,NaN,NaN,NaN,NaN,NaN,0,False
958,TOX-976,NaN,NaN,NaN,NaN,NaN,0,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts
85,TOX-91,calcium folinate,"calcium;(2S)-2-[[4-[(2-amino-5-formyl-4-oxo-3,...",135403647.0,Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)N[C@@H](CC...,KVUAALJSMIVURS-ZEDZUCNESA-L,1,True,True
232,TOX-239,GEMCITABINE HYDROCHLORIDE,"4-amino-1-[(2R,4R,5R)-3,3-difluoro-4-hydroxy-5...",60749.0,Cl.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)C2(F)F)c(=...,OKKDEIYWILRZIA-OSZBKLCCSA-N,1,True,True
349,TOX-359,NaN,"(7S,9S)-9-acetyl-7-[(2R,4S,5S,6S)-4-amino-5-hy...",62770.0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,GUGHGUXZJWAIAS-QQYBVWGSSA-N,1,True,True
350,TOX-360,Doxorubicin Hydrochloride,"(7S,9S)-7-[(2R,4S,5S,6S)-4-amino-5-hydroxy-6-m...",443939.0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,MWWSFMDVAYGXBV-RUELKSSGSA-N,1,True,True
445,TOX-455,NaN,4-amino-5-fluoro-3-[6-(4-methylpiperazin-1-yl)...,135611162.0,CC(O)C(=O)O.CN1CCN(c2ccc3c(c2)N/C(=C2/C(=O)N=c...,QDPVYZNVVQQULH-UHFFFAOYSA-N,1,True,True
626,TOX-640,Imatinib mesylate,methanesulfonic acid;4-[(4-methylpiperazin-1-y...,123596.0,CS(=O)(=O)O.Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc...,YLMAHDNUQAMNNX-UHFFFAOYSA-N,1,True,True
706,TOX-721,Vincristine sulfate,"methyl (1R,9R,10S,11R,12R,19R)-11-acetyloxy-12...",249332.0,CC[C@]1(O)C[C@H]2CN(CCc3c([nH]c4ccccc34)[C@@](...,AQTQHPDCURKLKT-JKDPCDLQSA-N,1,True,True
927,TOX-944,Rosiglitazone maleate,(Z)-but-2-enedioic acid;5-[[4-[2-[methyl(pyrid...,5281055.0,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1.O=C(O)...,SUFUKZSWUHZXAV-BTJKTKAUSA-N,1,True,True
939,TOX-956,Bendamustine hydrochloride,4-[5-[bis(2-chloroethyl)amino]-1-methylbenzimi...,77082.0,Cl.Cn1c(CCCC(=O)O)nc2cc(N(CCCl)CCCl)ccc21,ZHSKUOZOLHMKEA-UHFFFAOYSA-N,1,True,True
985,TOX-1003,Erlotinib hydrochloride,"N-(3-ethynylphenyl)-6,7-bis(2-methoxyethoxy)qu...",176871.0,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1.Cl,GTTBEUCJPZQMDZ-UHFFFAOYSA-N,1,True,True


#### Пропуски:

TAID                  0
Name                681
IUPAC Name          189
PubChem CID         187
Canonical SMILES     15
InChIKey            187
Toxicity Value        0
is_valid              0
has_salts             0
dtype: int64
TAID                 0.000000
Name                47.622378
IUPAC Name          13.216783
PubChem CID         13.076923
Canonical SMILES     1.048951
InChIKey            13.076923
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 6.36%


In [236]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Clinical_toxicity_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Clinical Toxicity_Clinical toxicity.csv', index=False)

Hepatotoxicity_Hepatotoxicity - химическое поражение печени, также известное как "токсическое заболевание печени" или "печеночная недостаточность, вызванная лекарствами"

In [237]:
df = pd.read_csv('../datasets/toxric_30_datasets/Hepatotoxicity_Hepatotoxicity.csv')
display(Markdown("### Химическое поражение печени:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value')

### Химическое поражение печени:

Размерность: (2889, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              2889 non-null   object 
 1   Name              1902 non-null   object 
 2   IUPAC Name        2808 non-null   object 
 3   PubChem CID       2809 non-null   float64
 4   Canonical SMILES  2847 non-null   object 
 5   InChIKey          2806 non-null   object 
 6   Toxicity Value    2889 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 158.1+ KB
None


#### Валидность молекул:

Корректных молекул: 2847
Некорректных молекул: 42
Доля невалидных: 1.45%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
3,TOX-27,NaN,NaN,NaN,NaN,NaN,0,False
117,TOX-530,NaN,NaN,NaN,NaN,NaN,0,False
302,TOX-1281,NaN,NaN,NaN,NaN,NaN,0,False
653,TOX-1632,NaN,NaN,NaN,NaN,NaN,1,False
724,TOX-1703,NaN,NaN,NaN,NaN,NaN,1,False
729,TOX-1708,NaN,NaN,NaN,NaN,NaN,1,False
870,TOX-1849,NaN,NaN,NaN,NaN,NaN,0,False
975,TOX-1954,NaN,NaN,NaN,NaN,NaN,0,False
1182,TOX-2161,NaN,NaN,NaN,NaN,NaN,1,False
1252,TOX-2231,NaN,NaN,NaN,NaN,NaN,0,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts
21,TOX-91,calcium folinate,"calcium;(2S)-2-[[4-[(2-amino-5-formyl-4-oxo-3,...",135403647.0,Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)N[C@@H](CC...,KVUAALJSMIVURS-ZEDZUCNESA-L,0,True,True
92,TOX-360,Doxorubicin Hydrochloride,"(7S,9S)-7-[(2R,4S,5S,6S)-4-amino-5-hydroxy-6-m...",443939.0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,MWWSFMDVAYGXBV-RUELKSSGSA-N,1,True,True
165,TOX-721,Vincristine sulfate,"methyl (1R,9R,10S,11R,12R,19R)-11-acetyloxy-12...",249332.0,CC[C@]1(O)C[C@H]2CN(CCc3c([nH]c4ccccc34)[C@@](...,AQTQHPDCURKLKT-JKDPCDLQSA-N,1,True,True
1408,TOX-2387,Hydroxyzine pamoate,4-[(3-carboxy-2-hydroxynaphthalen-1-yl)methyl]...,25096.0,O=C(O)c1cc2ccccc2c(Cc2c(O)c(C(=O)O)cc3ccccc23)...,ASDOKGIIKXGMNB-UHFFFAOYSA-N,1,True,True
1409,TOX-2388,methylene blue,[7-(dimethylamino)phenothiazin-3-ylidene]-dime...,6099.0,CN(C)c1ccc2nc3ccc(=[N+](C)C)cc-3sc2c1.[Cl-],CXKWCBBOMKCUKX-UHFFFAOYSA-M,1,True,True
...,...,...,...,...,...,...,...,...,...
2536,TOX-3515,DOXACURIUM CHLORIDE,"bis[3-[6,7,8-trimethoxy-2-methyl-1-[(3,4,5-tri...",60168.0,COc1cc(CC2c3c(cc(OC)c(OC)c3OC)CC[N+]2(C)CCCOC(...,APADFLLAXHIMFU-UHFFFAOYSA-L,0,True,True
2541,TOX-3520,NaN,[1-butoxy-3-[3-(3-butoxy-2-carbamoyloxypropyl)...,114965.0,CCC1(c2ccccc2)C(=O)NC(=O)NC1=O.CCCCOCC(CN1C(=O...,HYAMLWDZZVMLGY-UHFFFAOYSA-N,1,True,True
2545,TOX-3524,NaN,"3,6-diamino-N-[[15-amino-11-(2-amino-1,4,5,6-t...",73174934.0,CC1NC(=O)C(N)CNC(=O)C(C2CCN=C(N)N2)NC(=O)C(=CN...,VCOPTHOUUNAYKQ-UHFFFAOYSA-N,0,True,True
2862,TOX-109997,NaN,NaN,NaN,C=Cc1c(C)c2cc3nc(cc4[nH]c(cc5nc(cc1[nH]2)C(C)=...,NaN,1,True,True


#### Пропуски:

TAID                  0
Name                987
IUPAC Name           81
PubChem CID          80
Canonical SMILES     42
InChIKey             83
Toxicity Value        0
is_valid              0
has_salts             0
dtype: int64
TAID                 0.000000
Name                34.164071
IUPAC Name           2.803738
PubChem CID          2.769124
Canonical SMILES     1.453790
InChIKey             2.872966
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 50.71%


In [238]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")

# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Hepatotoxicity_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Hepatotoxicity_Hepatotoxicity.csv', index=False)

Respiratory Toxicity_Respiratory Toxicity - Токсичность для дыхательных путей

In [239]:
df = pd.read_csv('../datasets/toxric_30_datasets/Respiratory Toxicity_Respiratory Toxicity.csv')
display(Markdown("### Токсичность для дыхательных путей:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value')

### Токсичность для дыхательных путей:

Размерность: (1348, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              1348 non-null   object 
 1   Name              270 non-null    object 
 2   IUPAC Name        1161 non-null   object 
 3   PubChem CID       1195 non-null   float64
 4   Canonical SMILES  1336 non-null   object 
 5   InChIKey          1195 non-null   object 
 6   Toxicity Value    1348 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 73.8+ KB
None


#### Валидность молекул:

Корректных молекул: 1336
Некорректных молекул: 12
Доля невалидных: 0.89%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
12,TOX-1703,NaN,NaN,NaN,NaN,NaN,1,False
75,TOX-4564,NaN,NaN,NaN,NaN,NaN,1,False
152,TOX-8343,NaN,NaN,NaN,NaN,NaN,1,False
241,TOX-12424,NaN,NaN,NaN,NaN,NaN,1,False
274,TOX-14220,NaN,NaN,NaN,NaN,NaN,1,False
333,TOX-18139,NaN,NaN,NaN,NaN,NaN,1,False
374,TOX-22900,NaN,NaN,NaN,NaN,NaN,1,False
378,TOX-23815,NaN,NaN,NaN,NaN,NaN,1,False
657,TOX-56149,NaN,NaN,NaN,NaN,NaN,1,False
658,TOX-56150,NaN,NaN,NaN,NaN,NaN,1,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts
918,TOX-85612,Bergenin monohydrate,"(2R,3S,4S,4aR,10bS)-3,4,8,10-tetrahydroxy-2-(h...",201412.0,COc1c(O)cc2c(c1O)[C@@H]1O[C@H](CO)[C@@H](O)[C@...,QCWSXSAFDSGKAT-YOKSUNLASA-N,0,True,True
1050,TOX-90523,NaN,"6-benzyl-2,4-dimethyl-7-(4-methylphenyl)purino...",24839098.0,Cc1ccc(-c2cn3c4c(=O)n(C)c(=O)n(C)c4nc3n2Cc2ccc...,HACQDDATJNJBDK-UHFFFAOYSA-N,0,True,True
1055,TOX-90528,NaN,"(2R,4R)-1-[(2S)-5-(diaminomethylideneamino)-2-...",25062591.0,C[C@@H]1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCNC(=N)N...,AIEZTKLTLCMZIA-RZLLGALYSA-N,1,True,True
1335,TOX-112845,NaN,NaN,NaN,O.O.O.Oc1ccc2c3c1O[C@@H]1[C@@H](O)C=C[C@H]4[C@...,NaN,1,True,True


#### Пропуски:

TAID                   0
Name                1078
IUPAC Name           187
PubChem CID          153
Canonical SMILES      12
InChIKey             153
Toxicity Value         0
is_valid               0
has_salts              0
dtype: int64
TAID                 0.000000
Name                79.970326
IUPAC Name          13.872404
PubChem CID         11.350148
Canonical SMILES     0.890208
InChIKey            11.350148
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 60.76%


In [240]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Respiratory_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Respiratory Toxicity.csv', index=False)

Developmental Toxicity - это токсическое действие вещества на развитие потомства, которое приводит к структурным или функциональным нарушениям, проявляющимся с момента зачатия и до достижения взрослой жизни

In [241]:
df = pd.read_csv('../datasets/toxric_30_datasets/Developmental and Reproductive Toxicity_Developmental Toxicity.csv')
display(Markdown("### Токсическое действие вещества на развитие потомства:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value')

### Токсическое действие вещества на развитие потомства:

Размерность: (218, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              218 non-null    object 
 1   Name              133 non-null    object 
 2   IUPAC Name        188 non-null    object 
 3   PubChem CID       188 non-null    float64
 4   Canonical SMILES  188 non-null    object 
 5   InChIKey          187 non-null    object 
 6   Toxicity Value    218 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 12.1+ KB
None


#### Валидность молекул:

Корректных молекул: 188
Некорректных молекул: 30
Доля невалидных: 13.76%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
4,TOX-530,NaN,NaN,NaN,NaN,NaN,1,False
6,TOX-901,NaN,NaN,NaN,NaN,NaN,1,False
14,TOX-1632,NaN,NaN,NaN,NaN,NaN,1,False
15,TOX-1703,NaN,NaN,NaN,NaN,NaN,1,False
21,TOX-1954,NaN,NaN,NaN,NaN,NaN,1,False
24,TOX-2231,NaN,NaN,NaN,NaN,NaN,1,False
28,TOX-2394,NaN,NaN,NaN,NaN,NaN,0,False
30,TOX-2406,NaN,NaN,NaN,NaN,NaN,1,False
39,TOX-3152,NaN,NaN,NaN,NaN,NaN,1,False
42,TOX-3459,NaN,NaN,NaN,NaN,NaN,1,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts
27,TOX-2388,methylene blue,[7-(dimethylamino)phenothiazin-3-ylidene]-dime...,6099.0,CN(C)c1ccc2nc3ccc(=[N+](C)C)cc-3sc2c1.[Cl-],CXKWCBBOMKCUKX-UHFFFAOYSA-M,1,True,True
40,TOX-3163,AMARANTH,trisodium;3-hydroxy-4-[(4-sulfonatonaphthalen-...,13506.0,O=S(=O)([O-])c1ccc2c(N=Nc3ccc(S(=O)(=O)[O-])c4...,WLDHEUZGFKACJH-UHFFFAOYSA-K,1,True,True
41,TOX-3173,sodium chloride,sodium;chloride,5234.0,[Cl-].[Na+],FAPWRFPIFSIZLT-UHFFFAOYSA-M,1,True,True
43,TOX-3507,AMMONIUM CHLORIDE,azanium;chloride,25517.0,[Cl-].[NH4+],NLXLAEXVIDQMFP-UHFFFAOYSA-N,1,True,True
179,TOX-81706,SODIUM NITRATE,sodium;nitrate,24268.0,O=[N+]([O-])[O-].[Na+],VWDWKYIASSYTQR-UHFFFAOYSA-N,1,True,True
181,TOX-81860,Sodium acetate,sodium;acetate,517045.0,CC(=O)[O-].[Na+],VMHLLURERBWHNL-UHFFFAOYSA-M,0,True,True
183,TOX-82697,POTASSIUM NITRATE,potassium;nitrate,24434.0,O=[N+]([O-])[O-].[K+],FGIUAXJPYTZDNR-UHFFFAOYSA-N,1,True,True
194,TOX-84277,Sodium phenoxide,sodium;phenoxide,4445035.0,[Na+].[O-]c1ccccc1,NESLWCLHZZISNB-UHFFFAOYSA-M,0,True,True
195,TOX-84330,Docusate sodium,"sodium;1,4-bis(2-ethylhexoxy)-1,4-dioxobutane-...",23673837.0,CCCCC(CC)COC(=O)CC(C(=O)OCC(CC)CCCC)S(=O)(=O)[...,APSBXTVYXVQYAB-UHFFFAOYSA-M,1,True,True
196,TOX-84350,SODIUM NITRITE,sodium;nitrite,23668193.0,O=N[O-].[Na+],LPXPTNMVRIOKMN-UHFFFAOYSA-M,1,True,True


#### Пропуски:

TAID                 0
Name                85
IUPAC Name          30
PubChem CID         30
Canonical SMILES    30
InChIKey            31
Toxicity Value       0
is_valid             0
has_salts            0
dtype: int64
TAID                 0.000000
Name                38.990826
IUPAC Name          13.761468
PubChem CID         13.761468
Canonical SMILES    13.761468
InChIKey            14.220183
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 86.70%


In [242]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Developmental_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Developmental Toxicity.csv', index=False)

Reproductive Toxicity - это токсическое воздействие веществ, которое может негативно влиять на репродуктивную функцию взрослых мужчин и женщин, а также на развитие потомства. Это включает в себя снижение фертильности, нарушение работы половых органов, а также врожденные пороки и аномалии у детей. 

In [243]:
df = pd.read_csv('../datasets/toxric_30_datasets/Developmental and Reproductive Toxicity_Reproductive Toxicity.csv')
display(Markdown("### Токсическое действие вещества на репродуктивную функцию:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value').csv')


### Токсическое действие вещества на репродуктивную функцию:

Размерность: (156, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              156 non-null    object 
 1   Name              89 non-null     object 
 2   IUPAC Name        144 non-null    object 
 3   PubChem CID       146 non-null    float64
 4   Canonical SMILES  146 non-null    object 
 5   InChIKey          146 non-null    object 
 6   Toxicity Value    156 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 8.7+ KB
None


#### Валидность молекул:

Корректных молекул: 146
Некорректных молекул: 10
Доля невалидных: 6.41%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
3,TOX-1002,NaN,NaN,NaN,NaN,NaN,1,False
32,TOX-3927,NaN,NaN,NaN,NaN,NaN,1,False
34,TOX-4318,NaN,NaN,NaN,NaN,NaN,1,False
38,TOX-4430,NaN,NaN,NaN,NaN,NaN,1,False
67,TOX-5272,NaN,NaN,NaN,NaN,NaN,1,False
105,TOX-6747,NaN,NaN,NaN,NaN,NaN,1,False
127,TOX-81153,NaN,NaN,NaN,NaN,NaN,0,False
128,TOX-81338,NaN,NaN,NaN,NaN,NaN,1,False
142,TOX-85277,NaN,NaN,NaN,NaN,NaN,1,False
152,TOX-107504,NaN,NaN,NaN,NaN,NaN,1,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts
134,TOX-82413,AMMONIUM NITRATE,azanium;nitrate,22985.0,O=[N+]([O-])[O-].[NH4+],DVARTQFDIMZBAA-UHFFFAOYSA-O,1,True,True
140,TOX-84325,sodium benzoate,sodium;benzoate,517055.0,O=C([O-])c1ccccc1.[Na+],WXMKPNITSTVMEF-UHFFFAOYSA-M,1,True,True
143,TOX-85534,POTASSIUM NITRITE,potassium;nitrite,516910.0,O=N[O-].[K+],BXNHTSHTPBPRFX-UHFFFAOYSA-M,1,True,True


#### Пропуски:

TAID                 0
Name                67
IUPAC Name          12
PubChem CID         10
Canonical SMILES    10
InChIKey            10
Toxicity Value       0
is_valid             0
has_salts            0
dtype: int64
TAID                 0.000000
Name                42.948718
IUPAC Name           7.692308
PubChem CID          6.410256
Canonical SMILES     6.410256
InChIKey             6.410256
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 88.46%


In [244]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Reproductive_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Reproductive Toxicity.csv', index=False)

Irritation and Corrosion_Eye Corrosion - коррозия

In [246]:
df = pd.read_csv('../datasets/toxric_30_datasets/Irritation and Corrosion_Eye Corrosion.csv')
display(Markdown("### Токсическое действие на органы зрения приводящие к необратимым и серьёзным повреждениям:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value').csv')


### Токсическое действие на органы зрения приводящие к необратимым и серьёзным повреждениям:

Размерность: (2242, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              2242 non-null   object 
 1   Name              1311 non-null   object 
 2   IUPAC Name        2195 non-null   object 
 3   PubChem CID       2204 non-null   float64
 4   Canonical SMILES  2204 non-null   object 
 5   InChIKey          2201 non-null   object 
 6   Toxicity Value    2242 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 122.7+ KB
None


#### Валидность молекул:

Корректных молекул: 2204
Некорректных молекул: 38
Доля невалидных: 1.69%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
184,TOX-1708,NaN,NaN,NaN,NaN,NaN,1,False
402,TOX-2161,NaN,NaN,NaN,NaN,NaN,0,False
523,TOX-2394,NaN,NaN,NaN,NaN,NaN,1,False
900,TOX-3144,NaN,NaN,NaN,NaN,NaN,1,False
908,TOX-3247,NaN,NaN,NaN,NaN,NaN,0,False
1011,TOX-3927,NaN,NaN,NaN,NaN,NaN,1,False
1022,TOX-3950,NaN,NaN,NaN,NaN,NaN,0,False
1050,TOX-4048,NaN,NaN,NaN,NaN,NaN,0,False
1063,TOX-4101,NaN,NaN,NaN,NaN,NaN,0,False
1066,TOX-4115,NaN,NaN,NaN,NaN,NaN,0,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts


#### Пропуски:

TAID                  0
Name                931
IUPAC Name           47
PubChem CID          38
Canonical SMILES     38
InChIKey             41
Toxicity Value        0
is_valid              0
has_salts             0
dtype: int64
TAID                 0.000000
Name                41.525424
IUPAC Name           2.096343
PubChem CID          1.694915
Canonical SMILES     1.694915
InChIKey             1.828724
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 39.47%


In [251]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Corrosion_Eye_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Eye_Corrosion.csv', index=False)

Irritation and Corrosion_Eye Irritation - раздражение

In [249]:
df = pd.read_csv('../datasets/toxric_30_datasets/Irritation and Corrosion_Eye Irritation.csv')
display(Markdown("### Токсическое действие на органы зрения приводящие к обратимым повреждениям:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value').csv')


### Токсическое действие на органы зрения приводящие к обратимым повреждениям:

Размерность: (5125, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5125 entries, 0 to 5124
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              5125 non-null   object 
 1   Name              1870 non-null   object 
 2   IUPAC Name        5047 non-null   object 
 3   PubChem CID       5062 non-null   float64
 4   Canonical SMILES  5064 non-null   object 
 5   InChIKey          5053 non-null   object 
 6   Toxicity Value    5125 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 280.4+ KB
None


#### Валидность молекул:

Корректных молекул: 5064
Некорректных молекул: 61
Доля невалидных: 1.19%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid
50,TOX-530,NaN,NaN,NaN,NaN,NaN,1,False
165,TOX-1281,NaN,NaN,NaN,NaN,NaN,1,False
196,TOX-1632,NaN,NaN,NaN,NaN,NaN,1,False
429,TOX-2161,NaN,NaN,NaN,NaN,NaN,0,False
491,TOX-2288,NaN,NaN,NaN,NaN,NaN,1,False
...,...,...,...,...,...,...,...,...
4468,TOX-91804,NaN,NaN,NaN,NaN,NaN,1,False
4783,TOX-92119,NaN,NaN,NaN,NaN,NaN,1,False
4784,TOX-92120,NaN,NaN,NaN,NaN,NaN,1,False
4874,TOX-92211,NaN,NaN,NaN,NaN,NaN,0,False


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts


#### Пропуски:

TAID                   0
Name                3255
IUPAC Name            78
PubChem CID           63
Canonical SMILES      61
InChIKey              72
Toxicity Value         0
is_valid               0
has_salts              0
dtype: int64
TAID                 0.000000
Name                63.512195
IUPAC Name           1.521951
PubChem CID          1.229268
Canonical SMILES     1.190244
InChIKey             1.404878
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 74.77%


In [250]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Irritation_Eye_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Eye_Irritation.csv', index=False)

Кардиотоксичность - опасные побочные эффекты на сердце

In [252]:
df = pd.read_csv('../datasets/toxric_30_datasets/Cardiotoxicity_Cardiotoxicity-30.csv')
display(Markdown("### Кардиотоксичность:"))
print(f"Размерность: {df.shape}")
print(df.info())
total = len(df)

# ВАЛИДНОСТЬ МОЛЕКУЛ
df["is_valid"] = df["Canonical SMILES"].apply(is_valid_smiles)
valid = df["is_valid"].sum()
invalid = total - valid
display(Markdown("#### Валидность молекул:"))
print(f"Корректных молекул: {valid}")
print(f"Некорректных молекул: {invalid}")
print(f"Доля невалидных: {invalid / total:.2%}")
display(df[df["is_valid"] == False])

# НАЛИЧИЕ СОЛЕЙ
display(Markdown("#### Соли:"))
df["has_salts"] = df["Canonical SMILES"].apply(has_multiple_fragments)
with_salts = df["has_salts"].sum()
print(f"Молекул с солями: {with_salts}")
print(f"% молекул  солями: {with_salts / total:.2%}")
display(df[df["has_salts"] == True])

# ПРОПУСКИ
display(Markdown("#### Пропуски:"))
missing_count = df.isna().sum()
print(missing_count)
missing_percent = (100 * missing_count / len(df))
print(missing_percent)

# Распределение токсичности
display(Markdown("#### Распределение токсичности:"))
print(f"Процент токсичных в-в: {df['Toxicity Value'].mean():.2%}")
# sns.countplot(x='Toxicity Value', data=df, hue='Toxicity Value').csv')


### Кардиотоксичность:

Размерность: (1547, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              1547 non-null   object 
 1   Name              441 non-null    object 
 2   IUPAC Name        1291 non-null   object 
 3   PubChem CID       1295 non-null   float64
 4   Canonical SMILES  1547 non-null   object 
 5   InChIKey          1295 non-null   object 
 6   Toxicity Value    1547 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 84.7+ KB
None


#### Валидность молекул:

Корректных молекул: 1547
Некорректных молекул: 0
Доля невалидных: 0.00%


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid


...

,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value,is_valid,has_salts
159,TOX-2605,DIPHEMANIL METHYLSULFATE,"4-benzhydrylidene-1,1-dimethylpiperidin-1-ium;...",6126.0,COS(=O)(=O)[O-].C[N+]1(C)CCC(=C(c2ccccc2)c2ccc...,BREMLQBSKCSNNH-UHFFFAOYSA-M,1,True,True
380,TOX-83357,salmeterol xinafoate,2-(hydroxymethyl)-4-[1-hydroxy-2-[6-(4-phenylb...,56801.0,O=C(O)c1ccc2ccccc2c1O.OCc1cc(C(O)CNCCCCCCOCCCC...,XTZNCVSCVHTPAI-UHFFFAOYSA-N,1,True,True
488,TOX-92859,NaN,"2-benzhydryloxy-N,N-dimethylethanamine;8-chlor...",657227.0,CN(C)CCOC(c1ccccc1)c1ccccc1.Cn1c(=O)c2[nH]c(Cl...,NTZCUBODONTYCA-UHFFFAOYSA-M,1,True,True


#### Пропуски:

TAID                   0
Name                1106
IUPAC Name           256
PubChem CID          252
Canonical SMILES       0
InChIKey             252
Toxicity Value         0
is_valid               0
has_salts              0
dtype: int64
TAID                 0.000000
Name                71.493213
IUPAC Name          16.548158
PubChem CID         16.289593
Canonical SMILES     0.000000
InChIKey            16.289593
Toxicity Value       0.000000
is_valid             0.000000
has_salts            0.000000
dtype: float64


#### Распределение токсичности:

Процент токсичных в-в: 83.00%


In [253]:
# Обработка
df = df.dropna(subset=["Canonical SMILES"]).copy()
df['InChIKey'] = df['InChIKey'].fillna("unknown")
# Создание нового датасета
new_df = df[["InChIKey","Canonical SMILES","Toxicity Value"]].rename(columns={
    "Canonical SMILES" : "SMILES",
    "Toxicity Value" : "Cardiotoxicity-30_target"
}).copy()
new_df.to_csv('../datasets/toxric_processed/Cardiotoxicity-30.csv', index=False)